<a href="https://colab.research.google.com/github/barbosacesar/Intelig-ncia-Computacional/blob/main/Projeto_Disciplina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --no-cache-dir numpy==1.26.4 pandas ipywidgets opencv-python-headless fer matplotlib moviepy

In [2]:
!pip install deepface

In [3]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [4]:
# Montar Google Drive para acessar seus arquivos
from google.colab import drive
drive.mount('/content/drive')

# Mudar diretório para sua pasta de trabalho
%cd "/content/drive/MyDrive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [11]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript
import time
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import uuid
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import zipfile
import os

# Importando DeepFace
from deepface import DeepFace

EMOCOES_INFO = {
    'happy': ('Feliz', '😄', '#4CAF50'),
    'sad': ('Triste', '😢', '#1E88E5'),
    'angry': ('Bravo', '😠', '#E53935'),
    'surprise': ('Surpreso', '😲', '#FFEB3B'),
    'neutral': ('Neutro', '😐', '#9E9E9E'),
    'fear': ('Medo', '😨', '#8E24AA'),
    'disgust': ('Nojo', '🤢', '#6D4C41'),
}

def emocao_info(emocao):
    return EMOCOES_INFO.get(str(emocao).lower(), ('Desconhecida', '❓', '#000000'))

saida = widgets.Output()

def get_facial_attributes(filename, conf_threshold=0.3):
    try:
        result = DeepFace.analyze(img_path=filename, actions=['emotion'], enforce_detection=False)
        if isinstance(result, list):
            result = result[0]  # Se for lista, pega primeiro resultado
        emotions = result["emotion"]
        dominant_emotion = result["dominant_emotion"]
        confidence = emotions[dominant_emotion] / 100

        if confidence < conf_threshold:
            return []

        return [{
            "facial_expression": dominant_emotion,
            "confidence": confidence,
            "emotions": emotions
        }]
    except Exception as e:
        print("Erro na análise facial:", e)
        return []

def analisar_video(caminho_video, max_frames=10):
    cap = cv2.VideoCapture(caminho_video)
    contagem_emocoes = Counter()
    frame_count = 0

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Salvar frame temporariamente
        filename = f"/tmp/frame_{frame_count}.jpg"
        cv2.imwrite(filename, frame)

        # Analisar frame
        resultado = get_facial_attributes(filename)
        if resultado:
            emocao = resultado[0]["facial_expression"]
            contagem_emocoes[emocao] += 1

        frame_count += 1

    cap.release()
    return contagem_emocoes

# Funções mostrar_resultado, analisar_pasta_emocoes, on_upload_change_zip mantidas iguais

def on_upload_change(change):
    if not uploader.value:
        return

    arquivos = list(uploader.value.values())  # Pode enviar múltiplos arquivos
    contagem_emocoes = Counter()
    total_start = time.time()

    with saida:
        clear_output(wait=True)
        print(f"{len(arquivos)} arquivo(s) enviado(s). Iniciando análise...")

    for arquivo in arquivos:
        filename = arquivo['metadata']['name']
        content = arquivo['content']
        filepath = f"/tmp/{filename}"

        with open(filepath, 'wb') as f:
            f.write(content)

        # Se for ZIP, extrair e analisar imagens dentro
        if filename.lower().endswith('.zip'):
            pasta_temp = f"/tmp/extracted_{uuid.uuid4().hex}"
            os.makedirs(pasta_temp, exist_ok=True)
            try:
                with zipfile.ZipFile(filepath, 'r') as zip_ref:
                    zip_ref.extractall(pasta_temp)
            except Exception as e:
                print(f"Erro ao descompactar {filename}: {e}")
                continue

            imagens = list(Path(pasta_temp).rglob("*.jpg")) + list(Path(pasta_temp).rglob("*.jpeg")) + list(Path(pasta_temp).rglob("*.png"))
            if not imagens:
                print(f"Nenhuma imagem encontrada dentro do ZIP {filename}")
                continue

            for img_path in imagens:
                try:
                    resultado = get_facial_attributes(str(img_path))
                    if resultado:
                        emocao = resultado[0]["facial_expression"]
                        contagem_emocoes[emocao] += 1
                except Exception as e:
                    print(f"Erro ao analisar {img_path.name}: {e}")

        # Se for vídeo (avi, mp4, mov), analisar frames
        elif filename.lower().endswith(('.avi', '.mp4', '.mov')):
            try:
                contagem_video = analisar_video(filepath, max_frames=10)
                for emocao, count in contagem_video.items():
                    contagem_emocoes[emocao] += count
            except Exception as e:
                print(f"Erro ao analisar vídeo {filename}: {e}")

        # Se for imagem, analisar diretamente
        elif filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                resultado = get_facial_attributes(filepath)
                if resultado:
                    emocao = resultado[0]["facial_expression"]
                    contagem_emocoes[emocao] += 1
            except Exception as e:
                print(f"Erro ao analisar {filename}: {e}")

        else:
            print(f"{filename} ignorado (formato inválido).")

    total_end = time.time()

    with saida:
        clear_output(wait=True)
        total = sum(contagem_emocoes.values())
        if total == 0:
            print("Nenhuma emoção detectada nas imagens ou vídeos.")
            return

        print(f"Total de emoções detectadas: {total}")
        print(f"Tempo total: {total_end - total_start:.2f} segundos\n")

        print("Contagem por emoção:")
        for emocao, count in contagem_emocoes.items():
            pt, emoji, _ = emocao_info(emocao)
            print(f"{emoji} {pt:<10}: {count} ({count/total:.1%})")

        # Exibir gráfico
        try:
            labels = []
            values = []
            cores = []

            for emocao, count in contagem_emocoes.items():
                pt, emoji, cor = emocao_info(emocao)
                labels.append(f"{emoji} {pt}")
                values.append(count)
                cores.append(cor)

            plt.figure(figsize=(10, 5))
            bars = plt.bar(labels, values, color=cores)
            plt.title("Distribuição das Emoções Detectadas")
            plt.xlabel("Emoções")
            plt.ylabel("Frequência")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print("Erro ao gerar gráfico:", e)

    uploader.value.clear()

# Funções capture_image, capturar_webcam e mostrar_menu mantidas iguais, só corrigindo os tipos aceitos no uploader

def mostrar_menu():
    global uploader
    uploader = widgets.FileUpload(
        accept='.png,.jpg,.jpeg,.zip,.avi,.mp4,.mov',
        multiple=True,
        description='Enviar Arquivo',
        style={'button_color': '#2196F3', 'font_weight': 'bold', 'font_size': '16px'}
    )
    uploader.observe(on_upload_change, names='value')

    botao_webcam = widgets.Button(
        description="Capturar Webcam",
        button_style='info',
        layout=widgets.Layout(width='160px'),
        style={'font_weight': 'bold', 'font_size': '16px'}
    )
    botao_webcam.on_click(capturar_webcam)

    botoes = widgets.HBox([uploader, botao_webcam],
                         layout=widgets.Layout(justify_content='center', gap='20px', padding='10px'))

    display(HTML("<h2 style='text-align:center;'>Análise de Emoções Faciais</h2>"))
    display(HTML("<p style='text-align:center; font-size:16px;'>Envie uma imagem, vídeo ou capture da webcam</p>"))
    display(botoes)
    display(saida)

mostrar_menu()


Output()